# Dirichlet prior as database

BNLearner gives access of many priors for the parameters and structural learning. One of them is the Dirichlet prior which needs a a prior for every possible parameter in a BN. aGrUM/pyAgrum allows to use a database as a source of Dirichlet prior.

In [1]:
%matplotlib inline
from pylab import *
import matplotlib.pyplot as plt

import os

import pyAgrum as gum
import pyAgrum.lib.notebook as gnb


## generating a database for Dirichlet prior

In [10]:
bn = gum.fastBN("A->B->C")
gum.generateCSV(bn, "dirichlet.csv", 5000, with_labels=True)
bn

(gum::BayesNet<double>@0x55c26585ad60) BN{nodes: 3, arcs: 2, domainSize: 8, parameters: 10, compression ratio: -24% }

## generating a database for learning

In [11]:
bn2 = gum.fastBN("A;B;C")
gum.generateCSV(bn2, "database.csv", 8000, with_labels=True)
bn2

(gum::BayesNet<double>@0x55c265865fc0) BN{nodes: 3, arcs: 0, domainSize: 8, parameters: 6, compression ratio: 24% }

## Learning with Dirichlet prior (pour l'exemple j'apprends le Dirichlet avec la database comme Prior)

In [15]:
# bn is used to give the variables and their domains
learner = gum.BNLearner("dirichlet.csv", bn) 
learner.useScoreBIC() # or another score with no included prior
bn3=learner.learnBN()

def learnWithWeight(weight):
    # bn is used to give the variables and their domains
    learner = gum.BNLearner("dirichlet.csv", bn) 
    learner.setAprioriWeight(weight)
    learner.useAprioriDirichlet("database.csv")
    learner.useScoreBIC() # or another score with no included prior
    bn3=learner.learnBN()
    return bn3

weights=[0,1,5,10,1000,7000,100000]
gnb.sideBySide(bn3,*[learnWithWeight(w) for w in weights],
              captions=["without Dirichlet",*[f"with weight {w}" for w in weights]])

G <!-- A --> A A <!-- B --> B B <!-- C --> C C <!-- B->C --> B->C,G <!-- A --> A A <!-- B --> B B <!-- C --> C C <!-- B->C --> B->C,G <!-- A --> A A <!-- B --> B B <!-- C --> C C <!-- B->C --> B->C,G <!-- A --> A A <!-- B --> B B <!-- C --> C C <!-- B->C --> B->C,G <!-- A --> A A <!-- B --> B B <!-- C --> C C <!-- C->B --> C->B,G <!-- A --> A A <!-- B --> B B <!-- A->B --> A->B <!-- C --> C C <!-- C->B --> C->B,G <!-- A --> A A <!-- C --> C C <!-- A->C --> A->C <!-- B --> B B <!-- B->C --> B->C,G <!-- A --> A A <!-- C --> C C <!-- A->C --> A->C <!-- B --> B B <!-- B->C --> B->C
without Dirichlet,with weight 0,with weight 1,with weight 5,with weight 10,with weight 1000,with weight 7000,with weight 100000
